### **Mono Audio Feature Extraction**

In [27]:
from __future__ import print_function
import librosa
import librosa.display
import numpy as np
import pandas as pd
import os
from pydub import AudioSegment
from pydub.playback import play

In [28]:
file_path = 'Mono_Audio/Fall_Mono'
output_path = 'Mono_Audio_Feature/Fall_Feature'

fall_list = os.listdir(file_path)
count = 1

for file_name in fall_list:
    # 파일 확장자가 .wav인 경우에만 처리
    if file_name.endswith('.wav'):
        full_file_name = os.path.join(file_path, file_name)
        output_file_name = os.path.join(output_path, f'F{count}.csv')
        
        y, sr = librosa.load(full_file_name)
        duration = librosa.get_duration(y=y)
        onset_env = librosa.onset.onset_strength(y=y, sr=sr, hop_length=2205, aggregate=np.median)
        
        pd.set_option('display.max_columns', None)
        df = pd.DataFrame(onset_env)
        df.to_csv(output_file_name)
        
        count = count + 1

In [29]:
file_path = 'Mono_Audio/Non_Fall_Mono'
output_path = 'Mono_Audio_Feature/Non_Fall_Feature'

non_fall_list = os.listdir(file_path)
count = 1

for file_name in non_fall_list:
    # 파일 확장자가 .wav인 경우에만 처리
    if file_name.endswith('.wav'):
        full_file_name = os.path.join(file_path, file_name)
        output_file_name = os.path.join(output_path, f'NF{count}.csv')
        
        y, sr = librosa.load(full_file_name)
        duration = librosa.get_duration(y=y)
        onset_env = librosa.onset.onset_strength(y=y, sr=sr, hop_length=2205, aggregate=np.median)
        
        pd.set_option('display.max_columns', None)
        df = pd.DataFrame(onset_env)
        df.to_csv(output_file_name)
        
        count = count + 1

#### Combining Data

In [30]:
# %pip install audiomentations

In [31]:
# Function to read the second column from CSV files in a directory
def read_second_column_csv(directory):
    dataframes = []
    for filename in os.listdir(directory):
        if filename.endswith('.csv'):
            df = pd.read_csv(os.path.join(directory, filename), header=None, usecols=[1])
            flattened_df = df.T
            dataframes.append(flattened_df)
    return pd.concat(dataframes, ignore_index=True)

# # Function to apply various augmentations to audio data
# def augment_audio_data(y, sr, augmentations):
#     augmented_data = []
#     audio = AudioSegment(y, frame_rate=sr)
#     for aug in augmentations:
#         if aug == 'noise':
#             noise = AudioSegment.from_mono_audioarrays(np.random.normal(size=y.shape[0]), sr)
#             y_aug = audio.overlay(noise)
#         elif aug == 'shift':
#             shift_range = int(np.random.uniform(low=-5, high=5) * sr)
#             y_aug = audio[shift_range:] + audio[:shift_range]
#         elif aug == 'stretch':
#             rate = np.random.uniform(low=0.8, high=1.2)
#             y_aug = librosa.effects.time_stretch(y, rate=rate)
#         elif aug == 'pitch':
#             n_steps = np.random.uniform(low=-1, high=1)
#             y_aug = librosa.effects.pitch_shift(y, sr=sr, n_steps=n_steps)
#         elif aug == 'reverb':
#             y_aug = AudioSegment.from_mono_audioarray(y, sr)
#             y_aug = y_aug + y_aug.reverse()
#         elif aug == 'speed':
#             speed_rate = np.random.uniform(low=0.7, high=1.3)
#             y_aug = librosa.effects.time_stretch(y, speed_rate)
#         elif aug == 'change_volume':
#             change_in_dBFS = np.random.uniform(low=-10.0, high=10.0)
#             y_aug = AudioSegment.from_mono_audioarray(y, sr)
#             y_aug = y_aug + change_in_dBFS
#         augmented_data.append(y_aug)
#     return augmented_data


# # # Function to add augmentations to a dataset
# # def add_augmentations_fall(directory, sr=16000, augmentations=['shift'], repeats=10):
# #     augmented_dataframes = []
# #     for filename in os.listdir(directory):
# #         if filename.endswith('.csv'):
# #             df = pd.read_csv(os.path.join(directory, filename), header=None, usecols=[1])
# #             y = df.iloc[:, 0].values
# #             for _ in range(repeats):  # Repeat the augmentations
# #                 augmented_clips = augment_audio_data(y, sr, augmentations)
# #                 for aug_clip in augmented_clips:
# #                     aug_features = librosa.feature.mfcc(y=aug_clip, sr=sr, n_mfcc=13)
# #                     flattened_features = np.mean(aug_features, axis=1).reshape(1, -1)
# #                     augmented_dataframes.append(pd.DataFrame(flattened_features))
# #     return pd.concat(augmented_dataframes, ignore_index=True)
    

# # Function to add augmentations to a dataset
# def add_augmentations_non_fall(directory, sr=16000, augmentations=['noise', 'shift', 'stretch', 'pitch', 'reverb', 'speed', 'change_volume'], repeats=10):
#     augmented_dataframes = []
#     for filename in os.listdir(directory):
#         if filename.endswith('.csv'):
#             df = pd.read_csv(os.path.join(directory, filename), header=None, usecols=[1])
#             y = df.iloc[:, 0].values
#             for _ in range(repeats):  # Repeat the augmentations
#                 augmented_clips = augment_audio_data(y, sr, augmentations)
#                 for aug_clip in augmented_clips:
#                     aug_features = librosa.feature.mfcc(y=aug_clip, sr=sr, n_mfcc=13)
#                     flattened_features = np.mean(aug_features, axis=1).reshape(1, -1)
#                     augmented_dataframes.append(pd.DataFrame(flattened_features))
#     return pd.concat(augmented_dataframes, ignore_index=True)

In [32]:
from sklearn.utils import resample
from imblearn.over_sampling import KMeansSMOTE

In [33]:
fall_dir = 'Mono_Audio_Feature/Fall_Feature'
non_fall_dir = 'Mono_Audio_Feature/Non_Fall_Feature/'

In [34]:
# Read and combine CSV files
falls_data = read_second_column_csv(fall_dir)
not_falls_data = read_second_column_csv(non_fall_dir)

# # Apply augmentations
# # falls_augmented_data = add_augmentations_fall(fall_dir)
# not_falls_augmented_data = add_augmentations_non_fall(non_fall_dir)

# Combine original and augmented data
# combined_data = pd.concat([falls_data, not_falls_data, falls_augmented_data, not_falls_augmented_data])
combined_data = pd.concat([falls_data, not_falls_data])
combined_data['label'] = [1] * len(falls_data) + [0] * len(not_falls_data)
                        #  [0] * len(not_falls_augmented_data)
                        #  [1] * len(falls_augmented_data) + [0] * len(not_falls_augmented_data)

# outliers = detect_outliers_iqr(combined_data)
# combined_data[outliers] = np.nan

# Apply K-Means SMOTE for balancing
# kmeans_smote = KMeansSMOTE(random_state=123, k_neighbors=10, cluster_balance_threshold=0.1)
X = combined_data.drop('label', axis=1)
y = combined_data['label']
X = X.fillna(0)
# X_resampled, y_resampled = kmeans_smote.fit_resample(X, y)

# Combine resampled features and labels into a DataFrame
resampled_data = pd.DataFrame(X, columns=X.columns)
resampled_data['label'] = y

# Shuffle the dataset
resampled_data = resampled_data.sample(frac=1, random_state=123).reset_index(drop=True)

# Separate the dataset by label
falls_data_label = resampled_data[resampled_data['label'] == 1]
not_falls_data_label = resampled_data[resampled_data['label'] == 0]

# # Sample 350 entries from each subset
# sampled_falls_data = falls_data_label.sample(n=1200, random_state=123).reset_index(drop=True)
# sampled_not_falls_data = not_falls_data_label.sample(n=1200, random_state=123).reset_index(drop=True)

# Combine the sampled data
combined_sampled_data = pd.concat([falls_data_label, not_falls_data_label]).reset_index(drop=True)

# Shuffle the combined sampled dataset
semi_sampled_data = combined_sampled_data.sample(frac=1, random_state=123).reset_index(drop=True)

# Save to CSV
semi_sampled_data.to_csv('semi_dataset.csv', index=False)

In [35]:
semi_sampled_data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,label
0,0.0,0.0,7.740437,0.000000,0.000000,0.491012,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.00000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.00000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,0.0,0.0,7.079891,25.615664,8.616804,0.000000,9.816846,2.569111,0.000000,13.610187,2.206207,0.000000,0.000000,1.668438,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,19.331453,0.000000,3.462682,12.156923,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.00000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.00000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,0.0,0.0,1.811693,0.000000,0.000000,0.000000,0.000000,0.000000,30.208982,3.814112,3.607014,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.00000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.00000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,0.0,0.0,62.045340,0.000000,0.000000,1.097727,3.127792,0.000000,0.000000,0.000000,0.124878,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.00000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.00000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4,0.0,0.0,0.000000,0.000000,0.000000,53.713640,0.000000,0.000000,0.000000,11.898347,0.000000,1.195063,0.514551,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.00000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.